# Owl2vec_star_ext
In this process, we get ontology files and to change it to embedding model

In [1]:
import owl2vec4mappings

# Parameters:
# ontology_file1
# ontology_file2
# mapping_file
# config_file
# uri_doc
# lit_doc
# mix_doc
# but there are some parameters you can change in config file
gensim_model = owl2vec4mappings.extract_owl2vec_model(ontology_file1 = "./case_studies/Data/snomed.pharm.owl",
                                                ontology_file2 = './case_studies/Data/ncit.pharm.owl',
                                                mapping_file = './case_studies/Data/trainUnionS2Nph.tsv',
                                                config_file = "./default1.cfg",
                                                uri_doc = True, lit_doc = True, mix_doc = True)

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sev_s\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO: 1
INFO: 
 Access the ontology ...
INFO: There are 769113 triples in the ontology
* Owlready2 * Creating new ontology ncit.pharm <./case_studies/Data/ncit.pharm.owl#>.
* Owlready2 * ADD TRIPLE ./case_studies/Data/ncit.pharm.owl http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Ontology
* Owlready2 *     ...loading ontology ncit.pharm from ./case_studies/Data/ncit.pharm.owl...
* OwlReady2 * Importing 494284 object triples from ontology ./case_studies/Data/ncit.pharm.owl# ...
* OwlReady2 * Importing 658080 data triples from ontology ./case_studies/Data/ncit.pharm.owl# ...


* Owlready2 * Reseting property ann.use_in_alignment: new triples are now available.


* Owlready2 *     ...227 properties found: R100, R101, R102, R103, R104, R105, R106, R107, R108, R110, R111, R112, R113, R114, R115, R116, R122, R123, R124, R125, R126, R130, R131, R132, R133, R135, R136, R137, R138, R139, R140, R141, R142, R145, R146, R150, R153, R155, R40, R156, R158, R37, R159, R160, R163, R165, R166, R167, R168, R169, R170, R171, R172, R173, R174, R175, R176, R177, R178, R179, R180, R181, R182, R23, R24, R25, R26, R27, R28, R29, R30, R31, R32, R34, R35, R36, R38, R39, R41, R42, R44, R45, R46, R47, R48, R49, R50, R51, R52, R53, R54, R66, R72, R81, R82, R88, R89, comment, label, versionInfo, use_in_alignment, A1, A10, A11, A12, A13, A14, A15, A16, A17, A18, A19, A2, A20, A21, A22, A23, A24, A25, A26, A27, A28, A29, A3, A30, A33, A34, A4, A5, A6, A7, A8, A9, NHC0, P100, P101, P102, P106, P107, P108, P167, P171, P175, P200, P201, P203, P204, P205, P207, P208, P210, P211, P215, P216, P302, P310, P315, P316, P317, P319, P320, P321, P322, P325, P329, P330, P331, P332, P33

ValueError: This graph cannot be serialized to a strict format because there is no valid way to shorten http://snomed.info/id/736518005

In [ ]:
# Run
import configparser
config = configparser.ConfigParser()
config.read("default1.cfg")
output_folder=config['DOCUMENT']['cache_dir']

In [ ]:
#Gensim format
#Run
gensim_model.save(output_folder+"ontology"+'_'+config['DOCUMENT']['walk_depth']+".embeddings")

In [ ]:
#Txt format
#Run
gensim_model.wv.save_word2vec_format(output_folder+"ontology"+'_'+config['DOCUMENT']['walk_depth']+".embeddings.txt", binary=False)

## Loading embeddings and getting similarities

In [15]:
from gensim.models import KeyedVectors

#Embedding vectors generated above
model = KeyedVectors.load(output_folder+"ontology"+'_'+config['DOCUMENT']['walk_depth']+".embeddings", mmap='r')
wv = model.wv

vector = wv['committee']  # Get numpy vector of a word
print("Vector for 'committee'")
print(vector)
print()

#cosine similarity
similarity = wv.similarity('committee', 'http://conference#Committee')
print("Similarity between committee and http://conference#Committee", similarity)
print()

similarity = wv.similarity('http://conference#Reviewer', 'reviewer')
print(similarity)
print()

#Most similar cosine similarity
result = wv.most_similar(positive=['reviewer', 'committee'])
print(result)
print()

#Most similar entities: cosmul
result = wv.most_similar_cosmul(positive=['reviewer'])
print(result)
print()

INFO: loading Word2VecKeyedVectors object from ./caches_omim2ordo/ontology_3.embeddings
INFO: loading wv recursively from ./caches_omim2ordo/ontology_3.embeddings.wv.* with mmap=r
INFO: setting ignored attribute vectors_norm to None
INFO: loading vocabulary recursively from ./caches_omim2ordo/ontology_3.embeddings.vocabulary.* with mmap=r
INFO: loading trainables recursively from ./caches_omim2ordo/ontology_3.embeddings.trainables.* with mmap=r
INFO: setting ignored attribute cum_table to None
INFO: loaded ./caches_omim2ordo/ontology_3.embeddings


KeyError: "word 'committee' not in vocabulary"

In [ ]:
# Run
from gensim.models import Word2Vec
model = Word2Vec.load(output_folder+"ontology"+'_'+config['DOCUMENT']['walk_depth']+".embeddings")

In [ ]:
# Run
ranking_results = []
# save the scored candidate mappings in the same format as the original test.cands.tsv
f = open("scored.test.cands.tsv", "w")
f.write("SrcEntity" +'\t'+ "TgtEntity" +'\t'+ "TgtCandidates\n")
file = open(r"C:\Users\sev_s\Documents\OWL2Vec-Star\case_studies\Data\ncit2doid20223_test.cands.tsv")

for i in file.readlines()[1:]:
    src_ref_class = i.split('\t')[0]
    tgt_ref_class = i.split('\t')[1]
    tgt_cands = i.split('\t')[2]
    # print(src_ref_class, tgt_ref_class, tgt_cands)
    tgt_cands = eval(tgt_cands)  # transform string into list or sequence
    scored_cands = []
    for tgt_cand in tgt_cands:
        # assign a score to each candidate with an OM system
        try:
            matching_score = model.wv.similarity(src_ref_class, tgt_cand)
        except:
            matching_score = 0
        scored_cands.append([tgt_cand, matching_score])
    ranking_results.append([src_ref_class, tgt_ref_class, scored_cands])
    f.write(f'{src_ref_class}\t{tgt_ref_class}\t{scored_cands}\n')
print("done")

In [ ]:
# Run
import numpy as np
distance_results = []
# save the scored candidate mappings in the same format as the original test.cands.tsv
f = open("distance.test.cands.tsv", "w")
f.write("SrcEntity" +'\t'+ "TgtEntity" +'\t'+ "TgtCandidates\n")
file = open(r"C:\Users\sev_s\Documents\OWL2Vec-Star\case_studies\Data\ncit2doid20223_test.cands.tsv")

for i in file.readlines()[1:]:
    src_ref_class = i.split('\t')[0]
    tgt_ref_class = i.split('\t')[1]
    tgt_cands = i.split('\t')[2]
    # print(src_ref_class, tgt_ref_class, tgt_cands)
    tgt_cands = eval(tgt_cands)  # transform string into list or sequence
    scored_cands = []
    for tgt_cand in tgt_cands:
        # assign a score to each candidate with an OM system
        try:
            euc_dist = np.linalg.norm(model.wv[src_ref_class] - model.wv[tgt_cand])
        except:
            euc_dist = np.inf
        scored_cands.append([tgt_cand, euc_dist])
    distance_results.append([src_ref_class, tgt_ref_class, scored_cands])
    f.write(f'{src_ref_class}\t{tgt_ref_class}\t{scored_cands}\n')
print("done")

In [ ]:
# Run
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

# Load your Word2Vec model
model = Word2Vec.load(output_folder+"ontology"+'_'+config['DOCUMENT']['walk_depth']+".embeddings")

# Extract word vectors and corresponding words
words = list(model.wv.vocab.keys())
vectors = [model.wv[word] for word in words]

# Apply PCA to reduce dimensionality
num_dimensions = 2
pca = PCA(n_components=num_dimensions)
vectors_pca = pca.fit_transform(vectors)

# Elbow method for finding number of clusters

wcss = [] 
for i in range(1, 16): 
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(vectors_pca) 
    wcss.append(kmeans.inertia_)

plt.plot(range(1,16), wcss)

In [ ]:
# Apply KMeans clustering
num_clusters = 5  # You can change this based on your preference
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(vectors_pca)

# Visualize the clusters
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
for i in range(num_clusters):
    cluster_points = np.array([vectors_pca[j] for j in range(len(words)) if kmeans.labels_[j] == i])
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=colors[i], label=f'Cluster {i + 1}')

plt.title('Train-Val+ AML union LOgmap')
plt.legend()
plt.show()

In [ ]:
import random

# Choose a random index within the range of the words list
random_index = random.sample(range(len(words)), 30)

vector = {
    'X':vectors_pca[:,0],
    'Y':vectors_pca[:,1],
    'words': np.array(words),
    'label':kmeans.labels_
}

sample_vectors_pca = np.array([vector['X'][random_index], vector['Y'][random_index]]).T
sample_words = np.array(vector['words'][random_index])
sample_cluster = np.array(vector['label'][random_index])

# Visualize the clusters
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
markers = ['o', 'v', '^', '<', '>', 's', 'p', 'h', 'x', '+', 'd', '|', '_']

for i in range(num_clusters):
    cluster_points = np.array([sample_vectors_pca[j] for j in range(len(sample_words)) if sample_cluster[j] == i])
    plt.scatter(cluster_points[:,0], cluster_points[:,1], c=colors[i], marker=markers[i])

for i, txt in enumerate(sample_words):
    plt.annotate(txt, (sample_vectors_pca[i,0], sample_vectors_pca[i,1]), fontsize=5)